Instalações de bibliotecas necessárias:

OBS: A numpy possui um erro na biblioteca boruta_py, com isso é necessário abrir o arquivo borutapy e retirar todos os np.int, np.float, np.bool e converter para int, float e bool, sucessivamente.

In [1]:
'''!pip install pandas
!pip install sklearn
!pip install boruta
!pip install pgmpy
!pip install networkx
!pip install matplotlib'''

'!pip install pandas\n!pip install sklearn\n!pip install boruta\n!pip install pgmpy\n!pip install networkx\n!pip install matplotlib'

In [2]:
'''!pip install --upgrade --force-reinstall pgmpy'''


'!pip install --upgrade --force-reinstall pgmpy'

In [1]:
import pandas as pd
import time
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy
from sklearn.preprocessing import LabelEncoder
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import K2Score, BayesianEstimator

def boruta_feature_order(data_path, target_column):

    # Separar os dados em características (X) e alvo (y)
    X = data.drop(columns=[target_column])
    y = data[target_column]

    # Inicializar um classificador Random Forest
    rf = RandomForestClassifier(n_estimators=100, n_jobs=-1)

    # Inicializar o Boruta
    boruta_selector = BorutaPy(rf, n_estimators='auto', verbose=2)

    # Ajustar o Boruta aos dados
    boruta_selector.fit(X.values, y.values)
    print("Boruta inicializado com sucesso")

    selected_features = X.columns[boruta_selector.support_]
    unselected_features = X.columns[~boruta_selector.support_]

    all_features = [target_column] + list(selected_features) + list(unselected_features)
    df_reordered = data[all_features]

    return df_reordered

def k2(dataset, parents_nmax):
    estimator = K2Score(dataset)
    model = BayesianNetwork()
    nodes = list(dataset.columns)
    model.add_nodes_from(nodes)

    for i in range(1, len(nodes)):
        node = nodes[i]
        previous_nodes = nodes[:i]
        parents = []
        P_old = estimator.local_score(node, parents)
        proceed = True

        while proceed and (len(parents) < parents_nmax):
            candidates = list(set(previous_nodes) - set(parents))
            P_new = P_old

            for candidate in candidates:
                candidate_score = estimator.local_score(node, parents + [candidate])

                if candidate_score > P_new:
                    candidates_best = candidate
                    P_new = candidate_score

            if P_new > P_old:
                P_old = P_new
                parents.append(candidates_best)
                model.add_edge(candidates_best, node)
            else:
                proceed = False

    estrutura = list(model.edges)
    score = estimator.score(model)
    return estrutura, model, score

def tabular_cpd(model, data):
    for column in data.columns:
        data[column] = pd.Categorical(data[column])
    estimator = BayesianEstimator(model, data)
    cpds = [estimator.estimate_cpd(node) for node in model.nodes]
    return cpds

# Variáveis para acompanhar a melhor estrutura, melhor ordem, melhor score e a target_column relacionada
melhor_estrutura = None
melhor_ordem = None
melhor_score = float('-inf')
melhor_target = None

# Carregar os dados do CSV
data = pd.read_csv("hepartwo.csv")# caminho para o arquivo CSV
# Mapear os valores nominais para números inteiros únicos
data = data.apply(LabelEncoder().fit_transform)
variables = list(data.columns)

print("DataFrame original:")
print(list(data))



start_time = time.time()

# Loop através de cada coluna, considerando-a como a coluna alvo uma vez
for target_column in variables:
    if target_column == "target_column":
        continue

    df_reordered = boruta_feature_order(data, target_column)
    variable_target = list(df_reordered)
    estrutura, model, score = k2(df_reordered, 4)

    # Imprimir a ordem, estrutura e score para cada coluna alvo
    print(f'Ordem gerada com a feature {target_column}: {variable_target}')
    print(f'Estrutura gerada: {estrutura}')
    print(f'Score obtido: {score}')
    print()

    # Verificar se esta estrutura é a melhor até agora
    if score > melhor_score:
        melhor_estrutura = estrutura
        melhor_ordem = list(df_reordered)
        melhor_score = score
        melhor_target = target_column
        melhor_model = model

# Tabular as CPDs para o melhor modelo gerado
cpds = tabular_cpd(melhor_model, df_reordered)

end_time = time.time()
execution_time = end_time - start_time

# Abre o arquivo em modo de escrita
with open("hepartwo_best.txt", "w") as arquivo:
    # Escreve os prints no arquivo
    arquivo.write(f'Melhor ordem gerada com a feature ({melhor_target}): {melhor_ordem}\n')
    arquivo.write(f'Estrutura dessa ordem: {melhor_estrutura}\n')
    arquivo.write(f'Score obtido dessa ordem: {melhor_score}\n')
    arquivo.write(f'Tempo total de execução do algoritmo: {execution_time}\n')



c:\Users\Usuário\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DataFrame original:
['alcoholism', 'vh_amn', 'hepatotoxic', 'THepatitis', 'hospital', 'surgery', 'gallstones', 'choledocholithotomy', 'injections', 'transfusion', 'ChHepatitis', 'sex', 'age', 'PBC', 'fibrosis', 'diabetes', 'obesity', 'Steatosis', 'Cirrhosis', 'Hyperbilirubinemia', 'triglycerides', 'RHepatitis', 'fatigue', 'bilirubin', 'itching', 'upper_pain', 'fat', 'pain_ruq', 'pressure_ruq', 'phosphatase', 'skin', 'ama', 'le_cells', 'joints', 'pain', 'proteins', 'edema', 'platelet', 'inr', 'bleeding', 'flatulence', 'alcohol', 'encephalopathy', 'urea', 'ascites', 'hepatomegaly', 'hepatalgia', 'density', 'ESR', 'alt', 'ast', 'amylase', 'ggtp', 'cholesterol', 'hbsag', 'hbsag_anti', 'anorexia', 'nausea', 'spleen', 'consciousness', 'spiders', 'jaundice', 'albumin', 'edge', 'irregular_liver', 'hbc_anti', 'hcv_anti', 'palms', 'hbeag', 'carcinoma']
Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	69
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	69
Rejected: 	0
Iteration: 	3 / 100
C

In [2]:
from pgmpy.readwrite import XMLBIFWriter

# Especifique o caminho do arquivo onde deseja salvar o arquivo XMLBIF
file_path = "hepartwo_best.xmlbif"

# Adicione as CPDs ao modelo
for cpd in cpds:
    melhor_model.add_cpds(cpd)

# Escreva o modelo no formato XMLBIF
writer = XMLBIFWriter(melhor_model).write_xmlbif(file_path)

print(f"O arquivo XMLBIF foi gerado com sucesso em: {file_path}")


O arquivo XMLBIF foi gerado com sucesso em: hepartwo_best.xmlbif


TESTE:

1. Rodar 20x o K2 com ordem aleatória, para cada base de dados, e no fim tirar a média da função G (score), e o tempo;
    1.1 Guardar só a média das 20x;

2. Deixar anotado em uma planilha;

3. Rodar o K2 com o Boruta, para cada variável sendo classe, e a execução com o melhor G, eu guardo, guardando o tempo;

4. Guardar o XML;